# Brookes Invoice Match

### Material

In [2]:
import pandas as pd
import numpy as np
invoice = pd.read_csv("invoice.csv", encoding="latin1")
invoice = invoice[["Concatenate (Fusion)", "Concatenate (EBS)", "S. No.", "Supplier Name",
                  "Purchase Order No", "EBS No", "Fusion Receipt No", "Invoice Ref."]]
receiving = pd.read_csv("monitoring.csv",
                        encoding="latin1", parse_dates=["PO ( EBS ) Delivery Date","MRR Date ( EBS )",
                                                        "Sample Date", "MRR Release Date",
                                                        "Invoice Date", "Invoice Record Date"])
receiving = receiving.drop(receiving.tail(2).index) #remove last 2 rows from data frame
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )", "Invoice Number"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )", "Invoice Number"]].fillna("0")
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]].astype(int)
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]].astype(str)
receiving["Concatenate (EBS)"] = pd.concat([receiving['PO No  ( EBS )'].astype(str),
                                      receiving['MRR No. ( EBS )']], axis=1).agg('-'.join, axis=1)
receiving["Concatenate (Fusion)"] = pd.concat([receiving['PO No  ( EBS )'].astype(str),
                                      receiving['MRR No. ( Fusion )']], axis=1).agg('-'.join, axis=1)
required_invoices = receiving[receiving["Invoice Number"] == "0"]
required_invoices = required_invoices[['Concatenate (Fusion)', 'Concatenate (EBS)', 'Supplier','PO No  ( EBS )',
       'PO ( EBS ) Delivery Date', 'Quantity', 'Value', 'MRR No. ( EBS )',
       'MRR Date ( EBS )', 'MRR No. ( Fusion )', 'Item Name', 'MRR Release Date',
                       'Invoice Number', 'Invoice Date', 'Invoice Record Date']]
ebs_df = required_invoices.merge(invoice, how="left", on="Concatenate (EBS)")
ebs_df = ebs_df[["Concatenate (EBS)", "Supplier", "PO No  ( EBS )", "PO ( EBS ) Delivery Date", 
                   "MRR No. ( EBS )", "MRR Date ( EBS )", "MRR No. ( Fusion )", "MRR Release Date",
                   "Invoice Number", "Invoice Ref."]]
ebs_df["Invoice Ref."].fillna("Not Submitted", inplace=True)
ebs_df = ebs_df[(ebs_df["MRR Date ( EBS )"].dt.year == 2023)]
not_submitted_invoice = ebs_df[ebs_df["Invoice Ref."] == "Not Submitted"]
submitted_invoice = ebs_df[ebs_df["Invoice Ref."] != "Not Submitted"]
excel_file = "Invoices_data_ebs.xlsx"
with pd.ExcelWriter(excel_file, engine="xlsxwriter") as writer:
    submitted_invoice.to_excel(writer, sheet_name="Submitted", index=False)
    not_submitted_invoice.to_excel(writer, sheet_name="Not Submitted", index=False)
not_submitted_invoice.to_csv("required_invoices_ebs.csv", index=False)

### Services

In [4]:
import pandas as pd
import numpy as np
invoice = pd.read_csv("invoice.csv", encoding="latin1")
invoice = invoice[["Concatenate (Fusion)", "Concatenate (EBS)", "S. No.", "Supplier Name",
                  "Purchase Order No", "EBS No", "Fusion Receipt No", "Invoice Ref."]]
receiving = pd.read_csv("monitoring.csv",
                        encoding="latin1", parse_dates=["PO ( EBS ) Delivery Date","MRR Date ( EBS )",
                                                        "Sample Date", "MRR Release Date",
                                                        "Invoice Date", "Invoice Record Date"])
receiving = receiving.drop(receiving.tail(2).index) #remove last 2 rows from data frame
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )", "Invoice Number"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )", "Invoice Number"]].fillna("0")
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]].astype(int)
receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]] = receiving[["PO No  ( EBS )", "MRR No. ( EBS )", "MRR No. ( Fusion )"]].astype(str)
receiving["Concatenate (EBS)"] = pd.concat([receiving['PO No  ( EBS )'].astype(str),
                                      receiving['MRR No. ( EBS )']], axis=1).agg('-'.join, axis=1)
receiving["Concatenate (Fusion)"] = pd.concat([receiving['PO No  ( EBS )'].astype(str),
                                      receiving['MRR No. ( Fusion )']], axis=1).agg('-'.join, axis=1)

required_invoices = receiving[receiving["Invoice Number"] == "0"]
required_invoices = required_invoices[['Concatenate (Fusion)', 'Concatenate (EBS)', 'Supplier','PO No  ( EBS )',
       'PO ( EBS ) Delivery Date', 'Quantity', 'Value', 'MRR No. ( EBS )',
       'MRR Date ( EBS )', 'MRR No. ( Fusion )', 'Item Name', 'MRR Release Date',
                       'Invoice Number', 'Invoice Date', 'Invoice Record Date']]
fusion_df = required_invoices.merge(invoice, how="left", on="Concatenate (Fusion)")
fusion_df = fusion_df[["Concatenate (Fusion)", "Supplier", "PO No  ( EBS )", "PO ( EBS ) Delivery Date", 
                   "MRR No. ( EBS )", "MRR Date ( EBS )", "MRR No. ( Fusion )", "MRR Release Date",
                   "Invoice Number", "Invoice Ref."]]
fusion_df["Invoice Ref."].fillna("Not Submitted", inplace=True)
# fusion_df = fusion_df[(fusion_df["MRR Date ( EBS )"].dt.year == 2023)]
not_submitted_invoice = fusion_df[fusion_df["Invoice Ref."] == "Not Submitted"]
submitted_invoice = fusion_df[fusion_df["Invoice Ref."] != "Not Submitted"]
excel_file = "Invoices_data_fusion.xlsx"
with pd.ExcelWriter(excel_file, engine="xlsxwriter") as writer:
    submitted_invoice.to_excel(writer, sheet_name="Submitted", index=False)
    not_submitted_invoice.to_excel(writer, sheet_name="Not Submitted", index=False)
not_submitted_invoice.to_csv("required_invoices_fusion.csv", index=False)

In [8]:
# receiving["PO ( EBS ) Delivery Date"].fillna("2020-06-01", inplace=True)

# receiving.loc[:,("PO ( EBS ) Delivery Date","MRR Date ( EBS )", "Sample Date", "MRR Release Date",
#           "Invoice Date", "Invoice Record Date")] = receiving.loc[:,("PO ( EBS ) Delivery Date","MRR Date ( EBS )", "Sample Date", "MRR Release Date",
#           "Invoice Date", "Invoice Record Date")].fillna("0000-00-00")

In [12]:
# receiving["PO ( EBS ) Delivery Date"] = pd.to_datetime(receiving["PO ( EBS ) Delivery Date"])
# receiving["MRR Date ( EBS )"] = pd.to_datetime(receiving["MRR Date ( EBS )"])
# receiving["Sample Date"] = pd.to_datetime(receiving["Sample Date"])
# receiving["MRR Release Date"] = pd.to_datetime(receiving["MRR Release Date"])
# receiving["Invoice Date"] = pd.to_datetime(receiving["Invoice Date"])
# receiving["Invoice Record Date"] = pd.to_datetime(receiving["Invoice Record Date"])


In [1]:
!pip install cowsay

In [25]:
import cowsay
cowsay.fox('''Wishing You All A Very Happy New Year 2024!
Just For Fun In A Pythonic Way
From Atif Salam.....''')

  ___________________________________________
 /                                           \
| Wishing You All A Very Happy New Year 2024! |
| Just For Fun In A Pythonic Way              |
| From Atif Salam.....                        |
 \                                           /
                                            \
                                             \
                                              \
                                               |\_/|,,_____,~~`
                                               (.".)~~     )`~}}
                                                \o/\ /---~\\ ~}}
                                                  _//    _// ~}
